In [1]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

In [2]:
# Due to high RAM consumption when extracting features from all views together, the program was modified to take
# one view. Re-run the program by changing view numbers (i.e 1,6,10) and path to save features of all views.
def extract_data(filepath):
    tr_data1 = []
    te_data1 = []
    tr_data_labels = []
    te_data_labels = []
    
    IMG_SIZE = 100
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    vgg = tf.contrib.keras.applications.VGG16(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)
    vgg.trainable = False
    for label in sorted(os.listdir(filepath)):
        label_path = os.path.join(filepath, label)

        for files in sorted(os.listdir(label_path)):
            instances_path = os.path.join(label_path, files)

            if str(files) == 'test':
                for instance in sorted(os.listdir(instances_path)):
                    images_path = os.path.join(instances_path, instance)
                    te_data_labels.append(label)
                    for img in os.listdir(images_path):
                        image_path = os.path.join(images_path, img)
                        if int(img[0:-4]) == 10:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            te_data1.append(re_image)
                    
            elif str(files) == 'train':
                for instance in sorted(os.listdir(instances_path)):
                    images_path = os.path.join(instances_path, instance)
                    tr_data_labels.append(label)
                    for img in os.listdir(images_path):
                        image_path = os.path.join(images_path, img)
                        if int(img[0:-4]) == 10:
                            image = cv.imread(image_path)
                            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                            tr_data1.append(re_image)              
                
    tr_data1 = np.array(tr_data1, dtype="float") / 255.0
    te_data1 = np.array(te_data1, dtype="float") / 255.0
    
    tr_features = vgg.predict(tr_data1, batch_size=32)
    tr_features_flatten1 = tr_features.reshape((tr_features.shape[0], 3 * 3 * 512))

    te_features = vgg.predict(te_data1, batch_size=32)
    te_features_flatten1 = te_features.reshape((te_features.shape[0], 3 * 3 * 512))
    
    # data = data.reshape(data.shape[0], 50, 50, 3)
    tr_data_labels = np.array(tr_data_labels)
    te_data_labels = np.array(te_data_labels)
    label_names = np.unique(tr_data_labels)
    # data_labels = tf.one_hot(indices=data_labels, depth=10)
#     print(features_flatten.shape)
    return tr_features_flatten1, tr_data_labels, \
            te_features_flatten1, te_data_labels, label_names

In [3]:
if __name__ == "__main__":
       
    data_path = '../ModelNet40/image_dataset'

    # For easier disk read operation
    np_train_path1 = '../ModelNet40/extracted_features/train1.npy.gz'
    np_train_path2 = '../ModelNet40/extracted_features/train2.npy.gz'
    np_train_path3 = '../ModelNet40/extracted_features/train3.npy.gz'
    np_test_path1 = '../ModelNet40/extracted_features/test1.npy.gz'
    np_test_path2 = '../ModelNet40/extracted_features/test2.npy.gz'
    np_test_path3 = '../ModelNet40/extracted_features/test3.npy.gz'
    np_train_label_path = '../ModelNet40/extracted_features/train_labels.npy'
    np_test_label_path = '../ModelNet40/extracted_features/test_labels.npy'
    np_label_name_path = '../ModelNet40/extracted_features/label_names.npy'

    print("\n..........loading dataset from disk..........\n")
#     extract_data(data_path)
    train_data2, train_labels, \
    test_data2, test_labels, label_names = extract_data(data_path)

    os.makedirs(os.path.dirname(np_train_path2), exist_ok=True)

    with gzip.GzipFile(np_train_path3, "w") as f:
        np.save(f, train_data2)
    with gzip.GzipFile(np_test_path3, "w") as f:
        np.save(f, test_data2)

    np.save(np_train_label_path, train_labels)
    np.save(np_test_label_path, test_labels)
    np.save(np_label_name_path, label_names)


..........loading dataset from disk..........

Instructions for updating:
Colocations handled automatically by placer.
